In [6]:
import pandas as pd

model_csv = 'C:/Users/usuario/Desktop/Photo_Categorization/photoCategorization/data/results3/blip2_t5_pretrain_flant5xl_9322.csv'
df = pd.read_csv(model_csv)

Index(['Image_Id', 'Country', 'Region', 'People', 'Num_People', 'Role',
       'Gender', 'Category', 'Age_Categories', 'Travel_Group', 'Activity_Type',
       'Activity_Done', 'Context_Type', 'Resource_Types', 'Flora_Types',
       'Fauna_Types', 'Landscape_Types', 'Tourist_Resource_Types',
       'Heritage_Types', 'Cultural_Elements', 'Gastronomy_Types',
       'Leisure_Activities', 'Leisure_Types', 'Sports_Types',
       'Shopping_Facilities', 'Accommodation_Types', 'Infrastructure_Types'],
      dtype='object')


In [7]:
conteo_paises = df['Country'].value_counts()
num_paises_unicos = conteo_paises.nunique()

In [8]:
print(f"Total de países distintos: {num_paises_unicos}")
print(conteo_paises)

Total de países distintos: 5
Country
ESPAÑA     61
TURQUÍA    10
EGIPTO      9
CHIPRE      7
TURQUIA     7
MALTA       2
Name: count, dtype: int64


In [9]:
conteo_paises = df['Region'].value_counts()
num_paises_unicos = conteo_paises.nunique()

In [10]:
print(f"Total de países distintos: {num_paises_unicos}")
print(conteo_paises)

Total de países distintos: 9
Region
TÜRKISCHERIVIERA    15
GRANCANARIA         13
MALLORCA            13
LANZAROTE           12
TENERIFE            11
FUERTEVENTURA       10
EGIPTO               9
OTROS                8
TÜRKISCHEÄGÄIS       2
FUERTEVETURA         2
WESTKÜSTE            1
Name: count, dtype: int64


### Prueba de clasificación de eventuales

In [24]:
df_yes = df[df['People'].str.contains('yes', case=False, na=False)]

In [25]:
porcentajes = df_yes.groupby('Country').size() / df.groupby('Country').size() * 100
porcentajes = porcentajes.fillna(0)
print(porcentajes)
valor_min = porcentajes.min()
valor_max = porcentajes.max()

Country
CHIPRE     14.285714
EGIPTO     33.333333
ESPAÑA     26.229508
MALTA       0.000000
TURQUIA    28.571429
TURQUÍA    10.000000
dtype: float64


In [26]:
valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)
print(valores_normalizados)

Country
CHIPRE     0.428571
EGIPTO     1.000000
ESPAÑA     0.786885
MALTA      0.000000
TURQUIA    0.857143
TURQUÍA    0.300000
dtype: float64


In [27]:
porcentajes = df_yes.groupby('Region').size() / df.groupby('Region').size() * 100
porcentajes = porcentajes.fillna(0)
print(porcentajes)
valor_min = porcentajes.min()
valor_max = porcentajes.max()

Region
EGIPTO              33.333333
FUERTEVENTURA       40.000000
FUERTEVETURA         0.000000
GRANCANARIA         23.076923
LANZAROTE            8.333333
MALLORCA            53.846154
OTROS               12.500000
TENERIFE             9.090909
TÜRKISCHERIVIERA    20.000000
TÜRKISCHEÄGÄIS       0.000000
WESTKÜSTE            0.000000
dtype: float64


In [28]:
valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)
print(valores_normalizados)

Region
EGIPTO              0.619048
FUERTEVENTURA       0.742857
FUERTEVETURA        0.000000
GRANCANARIA         0.428571
LANZAROTE           0.154762
MALLORCA            1.000000
OTROS               0.232143
TENERIFE            0.168831
TÜRKISCHERIVIERA    0.371429
TÜRKISCHEÄGÄIS      0.000000
WESTKÜSTE           0.000000
dtype: float64


### Intento de automatización del proceso

In [29]:
import pandas as pd

#Creación de una opcion de automatizado por medio de indexión de tabla

model_csv = 'C:/Users/usuario/Desktop/Photo_Categorization/photoCategorization/data/results3/blip2_t5_pretrain_flant5xl_9322.csv'
df = pd.read_csv(model_csv)

In [30]:
#Falta termina la adición de las ultimas columnas 

condiciones = {
    'People': ['yes'],  
    'Role': ['primary', 'secondary'],          
    'Gender': ['male', 'female', 'other'],
    'Category': ['tourists', 'locals', 'workers'],
    'Age_Categories': ['children', 'young adults', 'adults', 'elderly'],
    'Travel_Group': ['family', 'couple'],
    'Activity_Type': ['active', 'passive'],
    'Context_Type': ['tourist attraction', 'leisure and recreation', 'lodging facility', 'natural resource', 'infrastructure'],
    'Resource_Types': ['flora', 'fauna', 'landscapes'],    
}

# Crear un DataFrame de combinaciones columna-variable
combinaciones = pd.DataFrame([
    {'Columna': col, 'Variable': var}
    for col, vars_ in condiciones.items()
    for var in vars_
])

In [31]:
def calcular_normalizado(df, columna, variable):
    # Filtrar filas que contienen la variable actual
    df_filtrado = df[df[columna].str.contains(variable, case=False, na=False)]

    # Calcular porcentajes por país
    porcentajes = df_filtrado.groupby('Country').size() / df.groupby('Country').size() * 100
    #porcentajes = df_filtrado.groupby('Region').size() / df.groupby('Region').size() * 100
    porcentajes = porcentajes.fillna(0)

    # Obtener los valores mínimo y máximo globales para normalizar
    valor_min = porcentajes.min()
    valor_max = porcentajes.max()

    # Calcular valores normalizados
    valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)

    return porcentajes, valores_normalizados

In [32]:
resultados_list = []
for _, row in combinaciones.iterrows():
    columna = row['Columna']
    variable = row['Variable']
    
    porcentajes, valores_normalizados = calcular_normalizado(df, columna, variable)
    resultados = pd.DataFrame({
        'Columna': columna,
        'Variable': variable,
        'País': porcentajes.index,
        'Porcentaje': porcentajes.values,
        'Valor Normalizado': valores_normalizados.values
    })
    resultados_list.append(resultados)

In [33]:
resultados_totales = pd.concat(resultados_list, ignore_index=True)
print(resultados_totales)

            Columna    Variable     País  Porcentaje  Valor Normalizado
0            People         yes   CHIPRE   14.285714           0.428571
1            People         yes   EGIPTO   33.333333           1.000000
2            People         yes   ESPAÑA   26.229508           0.786885
3            People         yes    MALTA    0.000000           0.000000
4            People         yes  TURQUIA   28.571429           0.857143
..              ...         ...      ...         ...                ...
145  Resource_Types  landscapes   EGIPTO   11.111111           0.847222
146  Resource_Types  landscapes   ESPAÑA   13.114754           1.000000
147  Resource_Types  landscapes    MALTA    0.000000           0.000000
148  Resource_Types  landscapes  TURQUIA    0.000000           0.000000
149  Resource_Types  landscapes  TURQUÍA   10.000000           0.762500

[150 rows x 5 columns]


In [34]:
resultados_totales.to_csv('resultados_normalizados_totales.csv', index=False)
print(resultados_totales.head())

  Columna Variable     País  Porcentaje  Valor Normalizado
0  People      yes   CHIPRE   14.285714           0.428571
1  People      yes   EGIPTO   33.333333           1.000000
2  People      yes   ESPAÑA   26.229508           0.786885
3  People      yes    MALTA    0.000000           0.000000
4  People      yes  TURQUIA   28.571429           0.857143
